In [1]:
# add all necessary imports
from tqdm import tqdm
from convokit import Corpus, Speaker, Utterance
from collections import defaultdict
import pandas as pd
import pprint as pp
from IPython.display import display

# 
from modules.DataPreprocesser import DataPreprocesser
#from modules.CorpusUtils import CorpusUtils

# Corpus Converter

This notebook takled the pre-processed dataframes and converts it to a Corpus Object

# Setting the Corpus Dataframes

In [3]:
filepath = "/Users/mishkin/Desktop/Research/Convo_Kit/ConvoKit_Disputes/data/preprocessed_dyads.csv"

final_data = DataPreprocesser(filepath)


display(final_data.getDataframe())



ValueError: malformed node or string on line 1: <ast.Name object at 0x147c91ed0>